In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import plotly.offline as pyo
import plotly.graph_objs as go
import seaborn as sns
from pycaret.classification import *

from sklearn.model_selection import GridSearchCV, KFold,KFold, cross_val_score

from sklearn.preprocessing import PolynomialFeatures, StandardScaler,RobustScaler 

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.linear_model import LogisticRegression,LinearRegression,Lasso

from sklearn.metrics import accuracy_score, f1_score,mean_squared_error,classification_report,confusion_matrix

from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline


import pycaret
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import logit, probit, poisson, ols
import statsmodels.formula.api as smf
from pycaret.regression import *

from statsmodels.formula.api import ols

from sklearn.model_selection import cross_val_score
from pandas_profiling import ProfileReport

from sklearn.linear_model import LinearRegression

import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.express as px

In [ ]:
from sklearn.metrics import r2_score
lin_model = make_pipeline(
    StandardScaler(),
    LinearRegression()
)

def lr_pred(df,target):
    cols = [c for c in df.columns if c!= target]
    X = df[cols]
    y = df[target].values
    
    model = lin_model.fit(X, y)
    
    y_pred = model.predict(X)
    
    mse = mean_squared_error(y, y_pred)
    
    r2 = r2_score(y,y_pred)
    
    dr = df.copy()
    dr['pred'] = y_pred
    
    return dr
    

def lr(df,target,cols):
    X = df[cols]
    y = df[target].values
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=0)
    model = lin_model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    
    r2 = r2_score(y_test,y_pred)
    
    return r2
def num_col(dmodel,type_of,rest):
    df = dmodel.copy()
    num = []
    for i in range(len((dmodel.dtypes == type_of).values)):
        v = (dmodel.dtypes == type_of)
        if v.values[i]:
            if v.index[i] not in rest:
                num.append(v.index[i])
    return num



def col_to_dobble_log(dmodel,type_of,rest,target):
    dr = col_to_log(dmodel,type_of,rest,target)
    df = dmodel.copy()
    
    num = num_col(dmodel,type_of,rest)
    
    df_log_log = np.log(np.log(df[num]+.0001)+0.0001)
    
    col = df_log_log.columns
    ncol = []
    for i in col:
        ncol.append(i+'_log_log')
    
    df_log_log.columns = ncol
    
    for i in df_log_log.columns:
        dr[i] = df_log_log[i]
    
    
    
    
    
    dll = dr.copy()
    
    cols_nan = []
    for i in range(len(np.isfinite(dll).sum() != len(dll))):
        v = (np.isfinite(dll).sum() != len(dll))[i]
        if v:
            cols_nan.append(i)
    Cols = []
    for i in range(len(dll.columns)):
        if i not in cols_nan:
            Cols.append(dll.columns[i])
    
    dll = dll[Cols]
    
    
    t1 = target + '_log'
    t2 = target + '_log_log'
    
    #print(t2,t2 in dll.columns)
    
    if t1 in dll.columns:
        #print('.....')
        if t2 in dll.columns:
            dll.drop([t1,t2],axis=1,inplace = True)
    
    
    
    return dll
def i_format(i):
    #print(i)
    if '_log_log' in i:
        return i[:i.index('_log_log')],i[:i.index('_log_log')+4],i
    
    else:
        if '_log' in i:
            return i[:i.index('_log')],i,i+'_log'
        
        if '_log' not in i:
            return i,i+'_log',i+'_log_log'

def forbiden_col(target,column_present):
    forbiden = [target]
    for i in column_present:
        #print(i)
        a,b,c = i_format(i)
        forbiden.append(a)
        forbiden.append(b)
        forbiden.append(c)
    return forbiden

def col_add(df,col_name,data):
    dfr = df.copy()
    dfr[col_name] = data
    return dfr

def best_farward_prop(df,target,column_present,p_r2):
    dr = df[column_present].copy()
    #print(dr.head())
    if len(column_present) < 3 :
        dr['dummy1'] = 1
        dr['dummy2'] = 2
        
    forbiden = forbiden_col(target,column_present)
    

    
    col = [c for c in df.columns if c not in forbiden]
    
    features = col.copy()
    
   
    Fwd = []
    R2 = []
    
    
    for i in features:
        dfr = col_add(dr,i,df[i])
        dfr_y = col_add(dfr,target,df[target])
        #return dfr_y,target
        #print(dfr_y.head())
        r2 = lr(dfr_y,target,[c for c in dfr_y.columns if c!= target])
        #print(r2)
        
        if r2 > p_r2:
            R2.append(r2)
            Fwd.append(i)
            #print(r2,i)
    
    if len(R2) > 0:
        #print(max(R2),Fwd[R2.index(max(R2))])
        return max(R2),Fwd[R2.index(max(R2))]
    
    else:
        return p_r2,''

def forward_prop(df,target,type_of,rest):
    
    dr = df.copy()
    column_present = [best_farward_prop(dr,target,[],0)[1]]
    
    forbiden = forbiden_col(target,column_present)
    
    
    
    y_pred = np.ones(len(df))*np.mean(df[target].values)
    
    best_r2 = r2_score(y_pred,df[target].values)
    print('initial r2 is ',best_r2)
    
    for i in dr.columns:
        if i not in forbiden:
            #print(column_present)
            p_r2,feature = best_farward_prop(dr,target,column_present,best_r2)
            if p_r2 > best_r2:
                best_r2 = p_r2
                column_present.append(feature)
                print(best_r2,feature)
            else:
                break
    return best_r2,column_present
        
        

def model_fwd(df,target,type_of,rest):
    dc_log_log = col_to_dobble_log(df,type_of,rest,target)
    return forward_prop(dc_log_log,target,type_of,rest)

def add_catef(df,dfcat):
    dr = df.copy()
    for i in dfcat.columns:
        dr[i] = dfcat[i]
    return dr

def features_to_drop_backward(df,target,rest):
    col = [c for c in df.columns if c != target]
    
    main_model_r2 = lr(df,target,col)
    best_r2 = main_model_r2
    
    features = col.copy()
    
    Drp = []
    R2 = []
    
    for i in features:
        if i not in rest:
            d_col = [c for c in features if c!= i]
            d_r2 = lr(df,target,d_col)

            if d_r2 > best_r2:
                Drp.append(i)
                R2.append(d_r2)
                #print(d_r2,i)
    
    if len(Drp) > 0:
        best = max(R2)
        droped = Drp[R2.index(best)]
        return best,droped
    else:
        return best_r2,''

def step_back(df,target,rest):
    
    dr = df.copy()
    
    col = [c for c in df.columns if c != target]
    main_model_r2 = lr(df,target,col)
    
    best_all = main_model_r2
    
    features = col.copy()
    
    for j in range(len(col)-len(rest)):
        r2,droped = features_to_drop_backward(dr,target,rest)
        if droped != '':
            print('droped column ',droped)
            dr.drop(droped,axis=1,inplace=True)
        if droped == '':
            break
    return best_all,r2,dr.columns

def step_backward_with_log(dmodel,target,type_of,rest):
    dll = col_to_log(dmodel,type_of,rest,target)
    
    t1 = target + '_log'
    
    if t1 in dll.columns:
        dll.drop([t1],axis=1,inplace = True)
    
    k = len(dll)

    cols_nan = []
    for i in range(len(np.isfinite(dll).sum() != k)):
        v = (np.isfinite(dll).sum() != k)[i]
        if v:
            cols_nan.append(i)

    Cols = []
    for i in range(len(dll.columns)):
        if i not in cols_nan:
            Cols.append(dll.columns[i])
    
    dll = dll[Cols]
    
    
    return step_back(dll,target,rest)

def step_backward_with_dubble_log(dmodel,target,type_of,rest):
    dll = col_to_dobble_log(dmodel,type_of,rest,target)
    
    t1 = target + '_log'
    t2 = target + '_log_log'
    
    if t1 in dll.columns:
        if t2 in dll.columns:
            dll.drop([t1,t2],axis=1,inplace = True)

    k = len(dll)
    
    cols_nan = []
    for i in range(len(np.isfinite(dll).sum() != k)):
        v = (np.isfinite(dll).sum() != k)[i]
        if v:
            cols_nan.append(i)

    Cols = []
    for i in range(len(dll.columns)):
        if i not in cols_nan:
            Cols.append(dll.columns[i])
    
    dll = dll[Cols]
    
    return step_back(dll,target,rest)


def step_backward_only(dll,target,type_of,rest):
    #dll = col_to_dobble_log(dmodel,type_of,rest,target)
    
    t1 = target + '_log'
    t2 = target + '_log_log'
    
    if t1 in dll.columns:
        if t2 in dll.columns:
            dll.drop([t1,t2],axis=1,inplace = True)

    k = len(dll)
    
    cols_nan = []
    for i in range(len(np.isfinite(dll).sum() != k)):
        v = (np.isfinite(dll).sum() != k)[i]
        if v:
            cols_nan.append(i)

    Cols = []
    for i in range(len(dll.columns)):
        if i not in cols_nan:
            Cols.append(dll.columns[i])
    
    dll = dll[Cols]
    
    return step_back(dll,target,rest)

def col_to_log(dmodel,type_of,rest,target):
    df = dmodel.copy()
    #df = df + 0.001
    
    num = num_col(dmodel,type_of,rest)
    
    df_log = np.log(df[num])
    
    col = df_log.columns
    ncol = []
    for i in col:
        ncol.append(i+'_log')
    
    df_log.columns = ncol
    
    for i in df_log.columns:
        df[i] = df_log[i]
        
    
    dll = df.copy()
    cols_nan = []
    for i in range(len(np.isfinite(dll).sum() != len(dll))):
        v = (np.isfinite(dll).sum() != len(dll))[i]
        if v:
            cols_nan.append(i)
    Cols = []
    for i in range(len(dll.columns)):
        if i not in cols_nan:
            Cols.append(dll.columns[i])
    
    dll = dll[Cols]
    
    t1 = target + '_log'
    
    if t1 in dll.columns:
        dll.drop([t1],axis=1,inplace = True)
    
    
    #dlog = pd.merge(df,df_log)
    return dll


In [ ]:
import geopy.distance
def dist (la1,ln1,la2,ln2):
    return geopy.distance.vincenty((la1,ln1), (la2,ln2)).km

# Collecting Macro data

In [ ]:
zip_pop = pd.read_csv(r'C:\Users\phani\Python Programming Codes Folder\Analytics Project\All_Macro_Data_Model\Rental_macro_merge_inc_osm\datasets_681859_1197071_plz_einwohner.csv')
osm = pd.read_csv(r'C:\Users\phani\Python Programming Codes Folder\Analytics Project\All_Macro_Data_Model\Rental_macro_merge_inc_osm\osm_data_macro_zip.csv')
macro = pd.read_csv(r'C:\Users\phani\Python Programming Codes Folder\Analytics Project\All_Macro_Data_Model\Rental_macro_merge_inc_osm\macro_zip_from_price_data_for_rent.csv')

macro = macro[['zip', 'DG', 'A_total_of_1_room', '1_or_2_rooms', '3_rooms', '4_rooms',
       '5_or_more_rooms', 'Total_population_of_age_18_25',
       'Total_populatio__of_age_25_30', 'Total_population__of_age_30_40',
       'Total_population_of_age_40_50', 'Deutsche', 'Foreigners',
       'Wage_and_income_taxpayers', 'Total_amount_of_income',
       'Wage_and_income_tax', 'gross_domestic_product',
       'Gross_domestic_product_per_person_in_employment',
       'Gross_domestic_product_per_inhabitant', 'amenity', 'bus',
       'public_transport', 'cafe', 'doctors', 'fast_food', 'hospital',
       'restaurant', 'zip_population', 'Population_State',
       'GDP_Per_Capita_state', 'Area_State', 'log_price', 'Lat_reg2',
       'Lng_reg2', 'Lat', 'Lng', 'lat_c', 'lng_c']]

# Collecting all the data to of rent and purchase saperately

In [ ]:
df1 = pd.read_csv('jun20_price_wohn_1_850.csv')
df2 = pd.read_csv('jun20_price_wohn_850_1690.csv')
frames = [df1,df2]
dp_whn = pd.concat(frames)

dp_whn.drop(['Unnamed: 0','timestamp'],axis=1,inplace=True)
dp_whn = dp_whn.drop_duplicates()

dr_whn = pd.read_csv(r'C:\Users\phani\Python Programming Codes Folder\Analytics Project\Apr20 all data\apr20_rental_no_duplicates.csv')
dr_whn.drop(['Unnamed: 0'],axis=1,inplace=True)
dr_whn = dr_whn.drop_duplicates()

dr_hau = pd.read_csv('jun20_rent_haus.csv')
dr_hau.drop(['Unnamed: 0','timestamp'],axis=1,inplace=True)
dr_hau = dr_hau.drop_duplicates()

dp_hau = pd.read_csv(r'C:\Users\phani\Python Programming Codes Folder\Analytics Project\Apr20 all data\apr20_price.csv')
dp_hau.drop(['Unnamed: 0'],axis=1,inplace=True)
dp_hau = dp_hau.drop_duplicates()

# Removing irrelevant columns (Price to Rent Ratio)

In [ ]:
dp_hau.drop(['obj_courtage',
 'obj_purchasePriceRange',
 'obj_constructionPhase',
 'obj_rented',
 'obj_international',
 'obj_usableArea',
 'obj_project_id',
 'obj_nbp',
 'Unnamed: 0.1'],axis=1,inplace=True)

dr_hau.drop(['obj_serviceCharge',

 'obj_hasKitchen',
 
 'obj_petsAllowed',
 'obj_depositLink',
 'obj_baseRentRange',
 'obj_assistedLiving',
 'obj_heatingCosts'],axis=1,inplace=True)

col_drop_r_hau = ['obj_telekomTvOffer', 'obj_cId','geo_bln',
       'obj_newlyConst','obj_picture',
       'obj_picturecount', 'obj_pricetrend', 'obj_telekomUploadSpeed','obj_telekomTrackingGroup',
       'obj_telekomInternetTechnology','obj_telekomInternetType', 'obj_pricetrendbuy', 'obj_scoutId',
       'obj_firingTypes','obj_telekomInternetProductName','obj_yearConstructedRange', 'obj_baseRent', 'obj_houseNumber',
       'obj_pricetrendrent','geo_krs','ga_cd_cxp_historicallisting',
       'obj_telekomDownloadSpeed', 'obj_street',
       'obj_telekomInternetUrlMobile', 'obj_telekomInternetUrl',
       'obj_streetPlain', 'geo_plz','obj_telekomHdTelephone', 'geo_land', 'ga_cd_via',
       'obj_telekomInternet', 'obj_immotype', 'obj_telekomInternetServices',
       'obj_telekomInternetProductAvailable', 'obj_cwId',
       'ga_cd_application_requirements','obj_noRoomsRange', 'ga_cd_maillead_default_shown', 'obj_objectnumber',
       'obj_livingSpaceRange','URL', 'beschreibung','obj_telekomHybridUploadSpeed', 'obj_groupnumber',
       'obj_telekomHybridDownloadSpeed', 'evt_count_pm_sig','ga_cd_developer_virtualreality', 'obj_galleryAd',
       'obj_energyType', 'obj_energyEfficiencyClass', 'obj_thermalChar',
       'ga_cd_customer_group', 'ga_cd_via_qualified','obj_ityp']

dr_hau.drop(col_drop_r_hau,axis=1,inplace=True)

c = []
for i in col_drop_r_hau:
    if i in dp_hau.columns:
        c.append(i)
dp_hau.drop(c,axis=1,inplace=True)

dp_hau.columns = ['obj_regio1', 'obj_heatingType', 'obj_plotAreaRange', 'obj_lotArea',
       'obj_yearConstructed', 'obj_noParkSpaces', 'obj_ExclusiveExpose',
       'obj_cellar', 'obj_livingSpace', 'zip', 'obj_condition',
       'obj_interiorQual', 'obj_noRooms', 'obj_purchasePrice',
       'obj_buildingType', 'obj_barrierFree', 'obj_regio3', 'obj_regio2',
       'obj_lastRefurbish', 'obj_numberOfFloors']

dr_hau.columns = ['obj_regio1', 'obj_heatingType', 'obj_plotAreaRange', 'obj_lotArea',
       'obj_totalRent', 'obj_yearConstructed', 'obj_ExclusiveExpose',
       'obj_cellar', 'obj_livingSpace', 'zip', 'obj_condition',
       'obj_interiorQual', 'obj_noRooms', 'obj_numberOfFloors',
       'obj_buildingType', 'obj_barrierFree', 'obj_regio3', 'obj_regio2',
       'obj_noParkSpaces', 'obj_lastRefurbish']

# Manual Removal of outliers - as many houses have rent as 0 and living space as 0

1. Living Space minimum was chosen as 20 (based on German Laws and Data Visualization - as they were mostly outliers)
2. Similarly removing outliers based on Visualizations done via Tableau

In [ ]:
price_house = dp_hau.copy()
rent_house = dr_hau.copy()

rent_house.drop(['obj_plotAreaRange'],axis=1,inplace=True)
price_house.drop(['obj_plotAreaRange'],axis=1,inplace=True)

price_house = price_house[price_house['obj_livingSpace'] > 20]
price_house = price_house[price_house['obj_livingSpace'] < 700]
price_house = price_house[price_house['obj_noRooms'] < 18]
price_house = price_house[price_house['obj_lotArea'] < 3000]

rent_house = rent_house[rent_house['obj_livingSpace'] > 20]
rent_house = rent_house[rent_house['obj_livingSpace'] < 700]
rent_house = rent_house[rent_house['obj_noRooms'] < 18]
rent_house = rent_house[rent_house['obj_lotArea'] < 3000]

rent_house = rent_house[rent_house['obj_totalRent'] > 100]
rent_house = rent_house[rent_house['obj_totalRent'] < 12000]

price_house = price_house[price_house['obj_purchasePrice'] > 60000]
price_house = price_house[price_house['obj_purchasePrice'] < 10000000]

price_house.drop(['obj_lastRefurbish','obj_numberOfFloors','obj_heatingType'],axis=1,inplace=True)
rent_house.drop(['obj_lastRefurbish','obj_numberOfFloors','obj_heatingType'],axis=1,inplace=True)

price_house.drop(['obj_noParkSpaces'],axis=1,inplace=True)
rent_house.drop(['obj_noParkSpaces'],axis=1,inplace=True)

# Some Helper functions to make pivot-tables
1. Pivot table created using median price and median rent (As price to rent ratio is defined on Median) - Source: Investopedia
2. since most of the macro features are either on a zip level or on a district level, and the data remains same in all the houses in a zip code, hence taking mean or median still remains the same.
3. For micro features the mean and median makes a difference, however mean and median is almost the same and there wasn't any significant change in the outputs of the models

In [ ]:
def pivot_basic_p_r(by,rent_house_macro,price_house_macro):
    d1 = pd.pivot_table(rent_house_macro, index=[by,'obj_buildingType'],aggfunc=np.median).reset_index(level=['obj_buildingType'])
    d2 = pd.pivot_table(price_house_macro, index=[by,'obj_buildingType'],aggfunc=np.median).reset_index(level=['obj_buildingType'])
    d3 = d1.merge(d2,on=by)
    d3 = d3[d3['obj_buildingType_x'] == d3['obj_buildingType_y']]
    
    ex = ['price_m2', 'anual_rent_m2']
    cols = ['obj_buildingType', 'Area_State', 'DG', 'Deutsche', 'Foreigners', 'GDP_Per_Capita_state', 'Gross_domestic_product_per_inhabitant', 'Gross_domestic_product_per_person_in_employment', 'Lat', 'Lat_reg2', 'Lng', 'Lng_reg2', 'Population_State', 'Total_amount_of_income', 'Total_populatio__of_age_25_30', 'Total_population__of_age_30_40', 'Total_population_of_age_18_25', 'Total_population_of_age_40_50', 'Wage_and_income_tax', 'Wage_and_income_taxpayers', 'gross_domestic_product', 'public_transport', 'restaurant', 'zip_population']
    #return d3
    
    c = []
    for i in d3.columns:
        if '_x' in i:
            i = i.replace('_x','')
        c.append(i)
    d3.columns = c
    
    for i in d3.columns:
        if '_y' in i:
            d3.drop(i,axis=1,inplace=True)
            
    de = ['log_price','1_or_2_rooms', '3_rooms', '4_rooms',
       '5_or_more_rooms', 'A_total_of_1_room',
       'obj_livingSpace', 'obj_lotArea', 'obj_noRooms', 'obj_totalRent','obj_purchasePrice','a_r']
    
    for i in de:
        if i in d3.columns:
            d3.drop(i,axis=1,inplace=True)
            
    d3['Price_Rent_ratio'] = d3['price_m2']/d3['anual_rent_m2']
    d3.drop(['price_m2','anual_rent_m2'],axis=1,inplace=True)
    
    #print(d3.columns)
    
    dc = d3.copy()

    dc['tot_population'] = dc['Deutsche'] + dc['Foreigners']
    dc['Foreigners_percent'] = 100*dc['Foreigners']/dc['tot_population']
    dc.drop(['Foreigners','Deutsche'],axis=1,inplace=True)


    change_col = ['Total_populatio__of_age_25_30', 'Total_population__of_age_30_40',
           'Total_population_of_age_18_25', 'Total_population_of_age_40_50']
    dc['all_age_pop'] = dc[change_col[0]] + dc[change_col[1]] + dc[change_col[2]] + dc[change_col[3]]
    dc['percent_pop_25_30'] = 100*(dc[change_col[0]]/dc['all_age_pop'])
    dc['percent_pop_18_25'] = 100*(dc[change_col[2]]/dc['all_age_pop'])
    dc['percent_pop_30_40'] = 100*(dc[change_col[1]]/dc['all_age_pop'])
    dc['percent_pop_40_50'] = 100*(dc[change_col[3]]/dc['all_age_pop'])

    dc.drop(change_col,axis=1,inplace=True)

    dc['population_density_state'] = dc['Population_State']/dc['Area_State']
    dc.drop(['Area_State','all_age_pop'],axis=1,inplace=True)
    
    nc = [c for c in dc.columns if c != 'obj_buildingType']
    dn = dc[nc]
    typ_h = pd.get_dummies(dc['obj_buildingType'],drop_first=True)
    dh = pd.concat([dn,typ_h],axis=1)
    
    
    dnlog = np.log(dn)
    c  = []
    for i in dnlog.columns:
        i = i + '_log'
        c.append(i)
    dnlog.columns = c
    if 'Price_Rent_Ratio_log' in dnlog.columns:
        dnlog.drop(['Price_Rent_Ratio_log'],axis=1,inplace=True)
    

    dr = dc.copy()
    for i in dnlog.columns:
        dr[i] = dnlog[i]
    dr = dr.dropna()
    if 'DG1_log' in dc.columns:
        dr.drop(['DG1_log'],axis=1,inplace=True)
    
    return dc,dn,dh,dr
    

# Feature Engineering
Creating new features from existing fearures so that it doesn't lead to problem of multi-coleniarity 

In [ ]:
price_house_macro = price_house.merge(macro)
rent_house_macro = rent_house.merge(macro)

price_whn = dp_whn.merge(macro)
rent_whn = dr_whn.merge(macro)



price_house_macro['price_m2'] = price_house_macro['obj_purchasePrice']/price_house_macro['obj_livingSpace']
price_house_macro['a_r'] = price_house_macro['obj_lotArea']/price_house_macro['obj_livingSpace']

rent_house_macro['anual_rent_m2'] = 12*rent_house_macro['obj_totalRent']/rent_house_macro['obj_livingSpace']
rent_house_macro['a_r'] = rent_house_macro['obj_lotArea']/rent_house_macro['obj_livingSpace']

price_house_macro = price_house_macro[price_house_macro['obj_buildingType'] != 'no_information']
rent_house_macro = rent_house_macro[rent_house_macro['obj_buildingType'] != 'no_information']


price_whn['area_per_room'] = price_whn['obj_livingSpace']/price_whn['obj_noRooms']
rent_whn['area_per_room'] = rent_whn['obj_livingSpace']/rent_whn['obj_noRooms']

price_hou = price_house_macro.drop(['obj_ExclusiveExpose', 'obj_cellar', 'obj_purchasePrice', 'obj_barrierFree', 'obj_regio3', 'obj_regio2', 'a_r'],axis=1)
rent_hou = rent_house_macro.drop(['obj_ExclusiveExpose', 'obj_cellar', 'obj_totalRent', 'obj_barrierFree', 'obj_regio3', 'obj_regio2', 'a_r'],axis=1)

price_hou.columns = ['obj_regio1', 'obj_lotArea', 'yearConstructed', 'obj_livingSpace',
       'zip', 'obj_condition', 'obj_interiorQual', 'obj_noRooms',
       'obj_buildingType', 'DG', 'A_total_of_1_room', '1_or_2_rooms',
       '3_rooms', '4_rooms', '5_or_more_rooms',
       'Total_population_of_age_18_25', 'Total_populatio__of_age_25_30',
       'Total_population__of_age_30_40', 'Total_population_of_age_40_50',
       'Deutsche', 'Foreigners', 'Wage_and_income_taxpayers',
       'Total_amount_of_income', 'Wage_and_income_tax',
       'gross_domestic_product',
       'Gross_domestic_product_per_person_in_employment',
       'Gross_domestic_product_per_inhabitant', 'amenity', 'bus',
       'public_transport', 'cafe', 'doctors', 'fast_food', 'hospital',
       'restaurant', 'zip_population', 'Population_State',
       'GDP_Per_Capita_state', 'Area_State', 'log_price', 'Lat_reg2',
       'Lng_reg2', 'Lat', 'Lng', 'lat_c', 'lng_c', 'price_m2']

rent_hou.columns = ['obj_regio1', 'obj_lotArea', 'yearConstructed', 'obj_livingSpace',
       'zip', 'obj_condition', 'obj_interiorQual', 'obj_noRooms',
       'obj_buildingType', 'DG', 'A_total_of_1_room', '1_or_2_rooms',
       '3_rooms', '4_rooms', '5_or_more_rooms',
       'Total_population_of_age_18_25', 'Total_populatio__of_age_25_30',
       'Total_population__of_age_30_40', 'Total_population_of_age_40_50',
       'Deutsche', 'Foreigners', 'Wage_and_income_taxpayers',
       'Total_amount_of_income', 'Wage_and_income_tax',
       'gross_domestic_product',
       'Gross_domestic_product_per_person_in_employment',
       'Gross_domestic_product_per_inhabitant', 'amenity', 'bus',
       'public_transport', 'cafe', 'doctors', 'fast_food', 'hospital',
       'restaurant', 'zip_population', 'Population_State',
       'GDP_Per_Capita_state', 'Area_State', 'log_price', 'Lat_reg2',
       'Lng_reg2', 'Lat', 'Lng', 'lat_c', 'lng_c', 'anual_rent_m2']

price_hou['area_per_room'] = price_hou['obj_livingSpace']/price_hou['obj_noRooms']
rent_hou['area_per_room'] = rent_hou['obj_livingSpace']/rent_hou['obj_noRooms']

price_hou.drop('obj_lotArea',axis=1,inplace=True)
rent_hou.drop('obj_lotArea',axis=1,inplace=True)


# Merging the all price data and all rental data

In [ ]:
rent_frames = [rent_whn,rent_hou]
price_frames = [price_whn,price_hou]

rent = rent_whn.append(rent_hou,ignore_index=True)
price = price_whn.append(price_hou,ignore_index=True)

# Removing outliers using KNN method - automated method using Pycaret

In [ ]:
from pycaret.anomaly import *
exp_ano = setup(rent)

knn = create_model('knn')
knn_df = assign_model(knn)
rent = knn_df[knn_df['Label'] == 0].drop(['Label','Score'],axis=1)

In [ ]:
from pycaret.anomaly import *
exp_ano = setup(price)

knn = create_model('knn')
knn_df = assign_model(knn)
price = knn_df[knn_df['Label'] == 0].drop(['Label','Score'],axis=1)

# Helper functions to create Pivot Table

In [ ]:
def pivot_merge_median_only(by,rent_house_macro,price_house_macro):
    d1 = pd.pivot_table(rent_house_macro, index=[by,'obj_buildingType'],aggfunc=np.median).reset_index(level=['obj_buildingType'])
    d2 = pd.pivot_table(price_house_macro, index=[by,'obj_buildingType'],aggfunc=np.median).reset_index(level=['obj_buildingType'])
    d3 = d1.merge(d2,on=by)
    d3 = d3[d3['obj_buildingType_x'] == d3['obj_buildingType_y']]
    
    d4 = d3.copy()
    
    d4['delta_year'] = d4['yearConstructed_x'] - d4['yearConstructed_y']
    d4['delta_area_per_room'] = d4['area_per_room_x'] - d4['area_per_room_y']
    d4['delta_livingspace'] = d4['obj_livingSpace_x'] - d4['obj_livingSpace_y']
    d4['delta_norooms'] = d4['obj_noRooms_x'] - d4['obj_noRooms_y']
    return d4

def pivot_merge_median(by,rent_house_macro,price_house_macro):
    d1 = pd.pivot_table(rent_house_macro, index=[by,'obj_buildingType'],aggfunc=np.median).reset_index(level=['obj_buildingType'])
    rent_sup = pd.pivot_table(rent_house_macro, index=[by,'obj_buildingType'],aggfunc='count').reset_index(level=['obj_buildingType'])['yearConstructed']
    d1['rent_supply'] = rent_sup
    
    d2 = pd.pivot_table(price_house_macro, index=[by,'obj_buildingType'],aggfunc=np.median).reset_index(level=['obj_buildingType'])
    price_sup = pd.pivot_table(price_house_macro, index=[by,'obj_buildingType'],aggfunc='count').reset_index(level=['obj_buildingType'])['yearConstructed']
    d2['price_sup'] = price_sup
    
    d3 = d1.merge(d2,on=by)
    d3 = d3[d3['obj_buildingType_x'] == d3['obj_buildingType_y']]
    
    d4 = d3.copy()
    
    d4['delta_year'] = d4['yearConstructed_x'] - d4['yearConstructed_y']
    d4['delta_area_per_room'] = d4['area_per_room_x'] - d4['area_per_room_y']
    d4['delta_livingspace'] = d4['obj_livingSpace_x'] - d4['obj_livingSpace_y']
    d4['delta_norooms'] = d4['obj_noRooms_x'] - d4['obj_noRooms_y']
    return d4

In [ ]:
d3 = pivot_merge_median('zip',rent,price)

# Using KNN to remove outliers - Pycaret Library

In [ ]:
from pycaret.anomaly import *
exp_ano = setup(d3)

knn = create_model('knn')
knn_df = assign_model(knn)
d3 = knn_df[knn_df['Label'] == 0].drop(['Label','Score'],axis=1)

# Manually removing some of the remaining outliers

In [ ]:
d3 = d3[d3['delta_year'] < 200]
d3 = d3[d3['delta_year'] > -200]

d3 = d3[d3['delta_norooms'] > -10]
d3 = d3[d3['delta_norooms'] < 10]


d3.drop(['delta_year','delta_area_per_room','delta_livingspace','delta_norooms'],axis=1,inplace=True)

In [ ]:
def pivot_refine(d4):
    d3 = d4.copy()
    ex = ['price_m2', 'anual_rent_m2']
    cols = ['obj_buildingType', 'Area_State', 'DG', 'Deutsche', 'Foreigners', 'GDP_Per_Capita_state', 'Gross_domestic_product_per_inhabitant', 'Gross_domestic_product_per_person_in_employment', 'Lat', 'Lat_reg2', 'Lng', 'Lng_reg2', 'Population_State', 'Total_amount_of_income', 'Total_populatio__of_age_25_30', 'Total_population__of_age_30_40', 'Total_population_of_age_18_25', 'Total_population_of_age_40_50', 'Wage_and_income_tax', 'Wage_and_income_taxpayers', 'gross_domestic_product', 'public_transport', 'restaurant', 'zip_population']
    #return d3
    
    d3['age'] = 2022 - .5*(d3['yearConstructed_x'] + d3['yearConstructed_y'])
    d3['rooms'] = .5*(d3['obj_noRooms_x'] + d3['obj_noRooms_y'])
    d3['livingSpace'] = .5*(d3['obj_livingSpace_x'] + d3['obj_livingSpace_y'])
    d3['avg_space_per_room'] = .5*(d3['area_per_room_x'] + d3['area_per_room_y'])
    
    d3.drop(['log_price_x','log_price_y','yearConstructed_x','yearConstructed_y','obj_noRooms_x','obj_noRooms_y','obj_livingSpace_x','obj_livingSpace_y','area_per_room_x','area_per_room_y'],axis=1,inplace=True)
    
    c = []
    for i in d3.columns:
        if '_x' in i:
            i = i.replace('_x','')
        c.append(i)
    d3.columns = c
    
    for i in d3.columns:
        if '_y' in i:
            d3.drop(i,axis=1,inplace=True)
            
    de = ['log_price','1_or_2_rooms', '3_rooms', '4_rooms',
       '5_or_more_rooms', 'A_total_of_1_room',
        'obj_totalRent','obj_purchasePrice','a_r']
    
    for i in de:
        if i in d3.columns:
            d3.drop(i,axis=1,inplace=True)

            
    d3['Price_Rent_ratio'] = d3['price_m2']/d3['anual_rent_m2']
    d3.drop(['price_m2','anual_rent_m2'],axis=1,inplace=True)
    
    #print(d3.columns)
    
    dc = d3.copy()



    change_col = ['Total_populatio__of_age_25_30', 'Total_population__of_age_30_40',
           'Total_population_of_age_18_25', 'Total_population_of_age_40_50']
    
    dc['population_18_30'] = dc['Total_populatio__of_age_25_30'] + dc['Total_population_of_age_18_25']
    dc['population_30_50'] = dc['Total_population__of_age_30_40'] + dc['Total_population_of_age_40_50']
    

    dc.drop(change_col,axis=1,inplace=True)

    dc['population_density_state'] = dc['Population_State']/dc['Area_State']
    dc.drop(['Area_State'],axis=1,inplace=True)
    
    dc['North_of_city'] = 111*(dc['Lat']-dc['lat_c'])
    dc['East_of_city'] = 111*(dc['Lng']-dc['lng_c'])
    
    dc.drop(['DG','amenity','lat_c','lng_c','Lat_reg2','Lng_reg2'],axis=1,inplace=True)
    dc = dc.fillna(dc.mean())
    
    #for i in ['cafe', 'doctors', 'fast_food', 'public_transport', 'restaurant']:
        #dc[i] = dc[i] + 1
    
    dc['dist_aprox'] = np.sqrt(np.square(dc['North_of_city']) + np.square(dc['East_of_city']))
    
    nc = [c for c in dc.columns if c not in ['obj_buildingType','North_of_city','East_of_city']]
    dn = dc[nc]
    typ_h = pd.get_dummies(dc['obj_buildingType'],drop_first=True)
    dh = pd.concat([dn,typ_h],axis=1)
    
    
    dnlog = np.log(dn)
    c  = []
    for i in dnlog.columns:
        i = i + '_log'
        c.append(i)
    dnlog.columns = c
    if 'Price_Rent_Ratio_log' in dnlog.columns:
        dnlog.drop(['Price_Rent_Ratio_log'],axis=1,inplace=True)
    

    dr = dc.copy()
    for i in dnlog.columns:
        dr[i] = dnlog[i]
    dr = dr.dropna()
    if 'DG1_log' in dc.columns:
        dr.drop(['DG1_log'],axis=1,inplace=True)
    
    dl = col_to_log(dn,'float64',[''],'Price_Rent_ratio')
    dlog = dl.merge(dc)
    
    '''
    r2,cl = forward_prop(dl,'Price_Rent_ratio','float64',[''])
    cl.append('Price_Rent_ratio')
    
    md = dlog[cl]
    dk = dc[['obj_buildingType','North_of_city','East_of_city']]
    
    md['in'] = list(range(len(md)))
    dk['in'] = list(range(len(dk)))
    


    dfwd = dk.merge(md,on='in')

    dfwd.drop('in',axis=1,inplace=True)
    '''

        
    return dc


# Adding More Macro information about the distance to nearest city and exporting to csv file

In [ ]:
dc = pivot_refine(d3)

dcc = dc.copy()

for i in range(len(city)):
    latv = 'lat_city'+str(i)
    lngv = 'lng_city'+str(i)
    dcc[latv] = city['lat_c'].iloc[i]
    dcc[lngv] = city['lng_c'].iloc[i]
    
for j in range(len(city)):
    d0 = []
    la0 = city['lat_c'].iloc[j]
    ln0 = city['lng_c'].iloc[j]
    for i in range(len(dcc)):
        lat2 = dcc['Lat'].iloc[i]
        lng2 = dcc['Lng'].iloc[i]
        d_city = dist(la0,ln0,lat2,lng2)
        d0.append(d_city)
    print(len(d0),len(dcc))
    var = 'dist_' + str(j)
    dcc[var] = d0

North = []
East = []
for i in range(len(dcc)):
    lt = dcc['Lat'].iloc[i]
    ln = dcc['Lng'].iloc[i]
    j = dcc['nearest_city_id'].iloc[i]
    ltc = dcc['lat_city'+str(j)].iloc[i]
    lnc = dcc['lng_city'+str(j)].iloc[i]
    north = 111*(lt-ltc)
    east = 111*(ln-lnc)
    North.append(north)
    East.append(east)
dcc['North_of_major_city'] = North
dcc['East_of_major_city'] = East


dcc['nearest_city_id'] = dcc[cdist].idxmin(axis=1)
dcc['nearest_city_id'] = dcc['nearest_city_id'].apply(lambda x: int(x.split('_')[1]))

cdist = []
for i in range(16):
    cdist.append('dist_'+str(i))

min_dist_city = dcc[cdist].min(axis=1)
dcc['nearest_major_city'] = min_dist_city

dc['nearest_major_city'] = dcc['nearest_major_city']
dc['North_of_major_city'] = dcc['North_of_major_city']
dc['East_of_major_city'] = dcc['East_of_major_city']

dc.drop(['dist_aprox','North_of_city',
       'East_of_city'],axis=1,inplace=True)

dc.to_csv('all_median_pr_merged_with_no_percent_for_elasticity.csv')